In [2]:
%pylab inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [1]:
import sys
import os
import os.path
import h5py
import tensorflow as tf
slim = tf.contrib.slim
losses = slim.losses
ops = slim.ops
import tfndlstm as ndlstm
import tfspecs as specs
from urllib2 import urlopen

In [3]:
url = "http://www.tmbdev.net/ocrdata-hdf5/mnist.h5"
if not os.path.exists("mnist.h5"):
    data = urlopen(url).read()
    with open("mnist.h5", "wb") as stream:
        stream.write(data)

In [17]:
from collections import namedtuple
Dataset = namedtuple("Dataset", "images classes size")

def loadh5(fname, prefix=""):
    h5 = h5py.File(fname)
    images = array(h5[prefix+"images"], "f")
    images.shape = images.shape + (1,)
    labels = array(h5[prefix+"labels"], "i")
    del h5
    print fname
    print images.shape, amin(images), amax(images)
    print labels.shape, amin(labels), amax(labels)
    return Dataset(images, labels, len(images))

train = loadh5("mnist.h5")
test = loadh5("mnist.h5", "test_")

mnist.h5
(60000, 28, 28, 1) 0.0 1.0
(60000,) 0 9
mnist.h5
(10000, 28, 28, 1) 0.0 1.0
(10000,) 0 9


In [18]:
with specs.ops:
    model = Cr(32, 3) | Mp(2) | Cr(64, 3) | Mp(2) | Flat | Fr(100) | Fl(10)

In [23]:
sess = tf.InteractiveSession()

inputs = tf.placeholder(tf.float32, [None, 28, 28, 1])
labels = tf.placeholder(tf.int32, [None])

outputs = model.funcall(inputs)

targets = tf.one_hot(labels, 10, 1.0, 0.0)
loss = tf.reduce_sum(tf.square(targets-tf.nn.sigmoid(outputs)))
optimizer = tf.train.AdamOptimizer(1e-4)
train_op = optimizer.minimize(loss)

errors = tf.not_equal(tf.argmax(outputs,1), tf.argmax(targets,1))
nerrors = tf.reduce_sum(tf.cast(errors, tf.float32))

sess.run(tf.initialize_all_variables())

In [24]:
def train_epoch(train=train, bs=20):
    total = 0
    count = 0
    for i in range(0, train.size, bs):
        batch_images = train.images[i:i+bs]
        batch_classes = train.classes[i:i+bs]
        feed_dict = {
            inputs: batch_images,
            labels: batch_classes,
        }
        k, _ = sess.run([nerrors, train_op], feed_dict=feed_dict)
        total += k
        count += len(batch_images)
    training_error = total * 1.0 / count
    return count, training_error

In [28]:
def evaluate(test=test):
    bs = 1000
    total = 0
    for i in range(0, test.size, bs):
        batch_images = test[0][i:i+bs]
        batch_classes = test[1][i:i+bs]
        feed_dict = {
            inputs: batch_images,
            labels: batch_classes,
        }
        k, = sess.run([nerrors], feed_dict=feed_dict)
        total += k
    test_error = total * 1.0 / test.size
    return total, test_error

In [ ]:
for epoch in range(100):
    _, training_err = train_epoch()
    _, testing_err = evaluate()
    print epoch, training_err, testing_err

0 0.0234666666667 0.02
1 0.0177166666667 0.0173
